In [1]:
!pip install julep

In [2]:
import os

import yaml
from julep import Julep
 
api_key = os.getenv("JULEP_API_KEY")


julep = Julep(api_key=api_key, environment="dev")

In [3]:
# Create the agent with error handling
try:
    agent = julep.agents.create(
        name="Julep Email Assistant",
        about=(
            "You are an agent that handles emails for julep users."
            + " Julep is a platform for creating kick-ass AI agents."
        ),
        model="gpt-4o",
        default_settings={"temperature": 0.2},
    )
    print("Agent created successfully!")
    print(f"Agent ID: {agent.id}")  # Print the created agent's ID

    # Retrieve the full agent details using the ID
    created_agent = julep.agents.get(agent.id)

    # Display the agent's details
    print(f"Agent Name: {created_agent.name}")
    print(f"Agent About: {created_agent.about}")
    print(f"Agent Model: {created_agent.model}")
    print(f"Agent Default Settings: {created_agent.default_settings}")

except Exception as e:
    print(f"Error creating or retrieving the agent: {e}")


Agent created successfully!
Agent ID: 2b61d5eb-e6a2-497d-9f3c-1d22edc03344
Agent Name: Julep Email Assistant
Agent About: You are an agent that handles emails for julep users. Julep is a platform for creating kick-ass AI agents.
Agent Model: gpt-4o
Agent Default Settings: DefaultSettings(frequency_penalty=0.0, length_penalty=1.0, min_p=0.01, presence_penalty=0.0, repetition_penalty=1.0, temperature=0.2, top_p=0.95)


In [4]:
agent.id

'2b61d5eb-e6a2-497d-9f3c-1d22edc03344'

In [5]:
mailgun_password = os.getenv("MAILGUN_PASSWORD")

task_def = yaml.safe_load(f"""
name: Julep Email Assistant

input_schema:
  type: object
  properties:
    from:
      type: string
    to:
      type: string
    subject:
      type: string
    body:
      type: string

tools:
- name: send_email
  integration:
    provider: email
    setup:
      host: smtp.mailgun.org
      password: {mailgun_password}
      port: 587
      user: postmaster@email.julep.ai

- name: search_docs
  system:
    resource: agent
    subresource: doc
    operation: search
    
main:
- prompt: |-
    You are {{{{ agent.name }}}}. {{{{ agent.about }}}}

    A user with email address {{{{ _.from }}}} has sent the following inquiry:
    ------
      Subject: {{{{ _.subject }}}}

      {{{{ _.body }}}}
    ------

    Can you generate a query to search the documentation based on this email?
    Just respond with the query as is and nothing else.

  unwrap: true

- tool: search_docs
  arguments:
    agent_id: "'{agent.id}'"
    text: _
    
- prompt: |-
    You are {{{{ agent.name }}}}. {{{{ agent.about }}}}

    A user with email address {{{{ inputs[0].from }}}} has sent the following inquiry:
    ------
      Subject: {{{{ inputs[0].subject }}}}

      {{{{ inputs[0].body }}}}
    ------

    Here are some possibly relevant snippets from the julep documentation:
    {{% for doc in _.docs %}}
      {{% for snippet in doc.snippets %}}
        {{{{ snippet.content }}}}
      {{% endfor %}}
    {{% endfor %}}
    ========

    Based on the above info, craft an email body to respond with as a json object.
    The json object must have `subject` and `body` fields.
  response_format:
    type: json_object
    
  unwrap: true
  
- evaluate:
    subject: "load_json(_.split('```json')[1].split('```')[0])['subject']"
    body: "load_json(_.split('```json')[1].split('```')[0])['body']"
    
- tool: send_email
  arguments:
    body: _.body
    from: "'postmaster@email.julep.ai'"
    subject: _.subject
    to: inputs[0]['from']
""")

In [6]:
task = julep.tasks.create(
    agent_id=agent.id,
    **task_def,
)

In [7]:
task.id

'9a687c54-a152-4513-bd70-28a72c136cdf'

In [8]:
execution = julep.executions.create(
    task_id=task.id,
    input={"from": "diwank@julep.ai", "to": "help@agents.new", "subject": "what's up", "body": "sup"},
)

In [9]:
julep.executions.get(execution.id)

Execution(id='a214a3c4-1dae-4f9d-b31e-a2b449aa7983', created_at=datetime.datetime(2024, 10, 11, 14, 47, 16, 774619, tzinfo=datetime.timezone.utc), input={'body': 'sup', 'from': 'diwank@julep.ai', 'subject': "what's up", 'to': 'help@agents.new'}, status='running', task_id='9a687c54-a152-4513-bd70-28a72c136cdf', updated_at=datetime.datetime(2024, 10, 11, 14, 47, 20, 144994, tzinfo=datetime.timezone.utc), error=None, metadata={}, output=None)

In [10]:
execution_transitions = julep.executions.transitions.list(
    execution_id=execution.id).items

for transition in reversed(execution_transitions):
    print("Type: ", transition.type)
    print("output: ", transition.output)
    print("-" * 100)

Type:  init
output:  {'body': 'sup', 'from': 'diwank@julep.ai', 'subject': "what's up", 'to': 'help@agents.new'}
----------------------------------------------------------------------------------------------------
Type:  step
output:  "what's up" site:julep.ai/docs
----------------------------------------------------------------------------------------------------
Type:  step
output:  {'docs': [], 'time': 0.00806879997253418}
----------------------------------------------------------------------------------------------------
Type:  step
output:  ```json
{
  "subject": "Hello!",
  "body": "Hi there! How can I assist you today? If you have any questions or need help with Julep, feel free to let me know!"
}
```
----------------------------------------------------------------------------------------------------
Type:  step
output:  {'body': 'Hi there! How can I assist you today? If you have any questions or need help with Julep, feel free to let me know!', 'subject': 'Hello!'}
------------